In [1]:
%load_ext autoreload
%autoreload 2

# Import Lib

In [2]:
import ezorm as ezo

In [3]:
from ezorm import DATABASE
DATABASE

'./db/ezorm.db'

# Create data model

In [4]:
class DPoPProofJTI(ezo.EzORM):
    jti:str

class AccessTokenJTI(DPoPProofJTI):
    client_id:str
    access_token:str
    exp:int
    active:bool = True
    remark:str = None
    
class RefreshTokenJTI(AccessTokenJTI):
    refresh_token:str

tables = [DPoPProofJTI, AccessTokenJTI, RefreshTokenJTI]

# Create tables

In [5]:
ezo.create_tables(tables)

Model: dpopproofjti created successfully
Model: accesstokenjti created successfully
Model: refreshtokenjti created successfully
All tables created successfully


In [6]:
query = """\
SELECT * 
FROM information_schema.tables 
WHERE table_schema = 'main';
""".strip()
ezo.execute(query, [], True)

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action,TABLE_COMMENT
0,ezorm,main,accesstokenjti,BASE TABLE,None,None,None,None,None,YES,NO,None,None
1,ezorm,main,dpopproofjti,BASE TABLE,None,None,None,None,None,YES,NO,None,None
2,ezorm,main,refreshtokenjti,BASE TABLE,None,None,None,None,None,YES,NO,None,None


# Read the table

In [7]:
from ezorm.crud import Read
query = Read(
    table=DPoPProofJTI,
    select=[DPoPProofJTI.jti],
).query()
print(query)
ezo.execute(query=query, data=[], response=True)

SELECT jti FROM dpopproofjti;


,jti


In [8]:
from ezorm.crud import Read
query = Read(
    table=AccessTokenJTI,
    select=[AccessTokenJTI.jti, AccessTokenJTI.client_id, AccessTokenJTI.access_token],
).query()
print(query)
ezo.execute(query=query, data=[], response=True)

SELECT jti, client_id, access_token FROM accesstokenjti;


,jti,client_id,access_token


In [9]:
from ezorm.crud import Read
query = Read(
    table=RefreshTokenJTI
).query()
print(query)
ezo.execute(query=query, data=[], response=True)

SELECT * FROM refreshtokenjti;


,jti,client_id,access_token,exp,active,remark,refresh_token


# Delete tables

In [10]:
ezo.delete_tables(tables)

Model: dpopproofjti deleted successfully
Model: accesstokenjti deleted successfully
Model: refreshtokenjti deleted successfully
All tables deleted successfully


In [11]:
query = """\
SELECT * 
FROM information_schema.tables 
WHERE table_schema = 'main';
""".strip()
ezo.execute(query, [], True)

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action,TABLE_COMMENT
